<a href="https://colab.research.google.com/github/Hktips/GEN_AI_Projects/blob/main/Transformer6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Layer, Embedding, Dense, LayerNormalization, Dropout
import numpy as np
def load_data(file_path):
  with open(file_path,'r') as f:
    data=f.read()
  return data
file_path="hp_1.txt"
text=load_data(file_path).lower()
#tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)
total_words=len(tokenizer.word_index)+1
input_sequences=[]
tokens=tokenizer.texts_to_sequences([text])[0]
seq_len=50
for i in range(seq_len,len(tokens)):
  input_sequences.append(tokens[i-seq_len:i+1])# Pad sequences and split inputs/targets
# after this X will have inputs and y will have label for those inputs
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_len + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# One-hot encode the labels
# note - there are other ways for encoding like pre-trained word2vec encoding and so on
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

{'<OOV>': 1, 'the': 2, '”': 3, 'and': 4, 'to': 5, 'a': 6, 'he': 7, 'of': 8, 'harry': 9, 'was': 10, 'it': 11, 'in': 12, 'his': 13, 'you': 14, 'said': 15, '—': 16, 'had': 17, 'on': 18, 'at': 19, 'that': 20, 'they': 21, 'as': 22, 'him': 23, 'i': 24, 'with': 25, 'but': 26, 'ron': 27, 'all': 28, 'out': 29, 'up': 30, 'for': 31, 'be': 32, 'them': 33, 'hagrid': 34, 'were': 35, 'have': 36, 'what': 37, 'there': 38, 'back': 39, 'hermione': 40, 'one': 41, 'from': 42, 'this': 43, 'not': 44, 'if': 45, 'she': 46, 'about': 47, 'into': 48, 'their': 49, 'been': 50, 'me': 51, 'know': 52, 'so': 53, 'off': 54, 'didn’t': 55, 'like': 56, 'could': 57, 'got': 58, 'get': 59, 'her': 60, 'down': 61, 'when': 62, 'over': 63, 'who': 64, 'looked': 65, '“i': 66, 'professor': 67, 'just': 68, 'very': 69, 'see': 70, 'is': 71, 'then': 72, 'no': 73, 'by': 74, 'do': 75, 'are': 76, 'your': 77, 'around': 78, 'now': 79, 'dumbledore': 80, 'he’d': 81, 'snape': 82, 'going': 83, 'an': 84, 'something': 85, 'through': 86, 'think': 8

# Transformer Model

In [7]:
class MultiHeadAttention(Layer):

  def __init__(self,embed_dim,num_heads):
      super(MultiHeadAttention,self).__init__()
      self.embed_dim = embed_dim#521
      self.num_heads = num_heads#8
      self.projection_dim=embed_dim//num_heads#64

      self.query_dense = tf.keras.layers.Dense(embed_dim)
      self.key_dense = tf.keras.layers.Dense(embed_dim)
      self.value_dense = tf.keras.layers.Dense(embed_dim)

      self.combine_heads= tf.keras.layers.Dense(embed_dim)
      self.combine_dense=tf.keras.layers.Dense(embed_dim)
  def attention(self,query,key,value):
      score=tf.matmul(query,key,transpose_b=True)#Q*k transpose
      score/=tf.math.sqrt(tf.cast(self.projection_dim,tf.float32))# #convert into float 32
      attention_probs=tf.nn.softmax(score,axis=-1)
      return tf.matmul(attention_probs,value)#probab *v
      #x can be any thing query and key and value
  def split_heads(self,x,batch_size):
    x=tf.reshape(x,(batch_size,-1,self.num_heads,self.projection_dim))
    return tf.transpose(x,perm=[0,2,1,3])
    #new shpe - batch,number of worrd(seq_len),number_head,projection
    #shape we want - batch_size number of word (seq_len) projection
    #batch_size of (8 heads of (4 word *64 dimenstion))
  def call(self,inputs):
    query,key,value=inputs
    batch_size=tf.shape(query)[0]
    query=self.split_heads(self.query_dense(query),batch_size)
    key=self.split_heads(self.key_dense(key),batch_size)
    value=self.split_heads(self.value_dense(value),batch_size)

    attention=self.attention(query,key,value)
    attention=tf.transpose(attention,perm=[0,2,1,3])
    concat_attention=tf.reshape(attention,(batch_size,-1,self.embed_dim))
    return self.combine_heads(concat_attention)


In [8]:
class TransformerBlock(Layer):
  def __init__(self,embed_dim,num_heads,ff_dim,rate=0.1):
    super(TransformerBlock,self).__init__()
    self.att=MultiHeadAttention(embed_dim,num_heads)

    self.ffn=tf.keras.Sequential([
        Dense(ff_dim,activation="relu"),
        Dense(embed_dim)
    ])
    #(x-mean)/root(variance+epotion)
    self.layernorm1=LayerNormalization(epsilon=le-6)
    self.layernorm2=LayerNormalization(epsilon=le-6)
    self.dropout1=Dropout(rate)
    self.dropout2=Dropout(rate)
  def call (self,input,trainig):
    attn_output=self.att(input)
    attn_output=self.dropout(att_output,training=training)
    out1=self.layernormal(inputs + attn_output)#passing input and output
    ffn_output=self.ffn(out1)
    ffn_output=self.dropout2(ffn_output,training=trainig)
    return self.layernorm2(out1+ffn_output)#Residual connection


In [9]:
from tensorflow.keras.layers import Layer
class TokenAndPositionEmbedding(Layer):
  def __init__(self,maxlen,vocab_size,embed_dim):
    super(TokenAndPositionEmbedding,self).__init__()
    self.token_emb=Embedding(input_dim=vocab_size,output_dim=embed_dim)
    self.pos_emb=Embedding(input_dim=maxlen,output_dim=embed_dim)
  def call(self,x):
    maxlen=tf.shape(x)[-1]
    positions=tf.range(start=0,limit=maxlen,delta=1)
    positions=self.pos_emb(positions)
    x=self.token_emb(x)
    return x+positions



In [23]:
# Model Parameters
embed_dim = 128    # Embedding size
num_heads = 4      # Number of attention heads
ff_dim = 512       # Feed-forward layer size
maxlen = seq_len # here it is 50 defined above
# below total words = 6662 (see above — basically all tokens in the text)
# Build the model
inputs = tf.keras.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, total_words, embed_dim)
x = embedding_layer(inputs)
print(x.shape)

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x, training=True)
print(x.shape)

x = x[:, -1, :]
print(x.shape)

x = Dense(total_words, activation="softmax")(x)
print(x.shape)

model = tf.keras.Model(inputs=inputs, outputs=x)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()


(None, 50, 128)


NameError: name 'le' is not defined

In [ ]:
'''class MultiHeadAttention(Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim  # e.g., 521
        self.num_heads = num_heads  # e.g., 8
        self.projection_dim = embed_dim // num_heads  # e.g., 64

        # Dense layers to create queries, keys, and values
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)

        # Final dense layer to combine the attention heads
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        # Scaled dot-product attention
        score = tf.matmul(query, key, transpose_b=True)
        score /= tf.math.sqrt(tf.cast(self.projection_dim, tf.float32))
        attention_weights = tf.nn.softmax(score, axis=-1)
        return tf.matmul(attention_weights, value)

    def split_heads(self, x, batch_size):
        # Reshape into (batch_size, num_heads, seq_len, projection_dim)
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value = inputs
        batch_size = tf.shape(query)[0]

        # Linear projections
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Split into multiple heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Attention
        attention_output = self.attention(query, key, value)

        # Combine heads back
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, -1, self.embed_dim))

        # Final linear layer
        output = self.combine_heads(concat_attention)
        return output
        '''


NameError: name 'Layer' is not defined